## EY Data Challenge - NY Mesonet Weather Dataset

## Load In Dependencies

In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
import geopandas as gpd

# Geospatial operations
import rasterio
from rasterio import windows  
from rasterio import features  
from rasterio import warp
from rasterio.warp import transform_bounds 
from rasterio.windows import from_bounds 

# Image Processing
from PIL import Image

# Coordinate transformations
from pyproj import Proj, Transformer, CRS

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm

## Predictor Variable

In [2]:
# Load the training data from csv file and display the first few rows to inspect the data
Bronx_weather_df = pd.read_excel("NY_Mesonet_Weather.xlsx", sheet_name = "Bronx")
Bronx_weather_df["location"] = "Bronx"
# Convert 'Date / Time' in weather_df to standard datetime format (removing timezone)
Bronx_weather_df.rename(columns={'Date / Time': 'datetime'}, inplace=True)
Bronx_weather_df['datetime'] = pd.to_datetime(Bronx_weather_df['datetime'], errors='coerce').dt.tz_localize(None)
Bronx_weather_df.head()

,datetime,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2],location
0,2021-07-24 06:00:00,19.3,88.2,0.8,335,12,Bronx
1,2021-07-24 06:05:00,19.4,87.9,0.8,329,18,Bronx
2,2021-07-24 06:10:00,19.3,87.6,0.7,321,25,Bronx
3,2021-07-24 06:15:00,19.4,87.4,0.5,307,33,Bronx
4,2021-07-24 06:20:00,19.4,87.0,0.2,301,42,Bronx


In [3]:
# Display rows that are duplicates
duplicate_rows_bronx = Bronx_weather_df[Bronx_weather_df.duplicated()]
print(duplicate_rows_bronx)


Empty DataFrame
Columns: [datetime, Air Temp at Surface [degC], Relative Humidity [percent], Avg Wind Speed [m/s], Wind Direction [degrees], Solar Flux [W/m^2], location]
Index: []


In [4]:
Manhattan_weather_df = pd.read_excel("NY_Mesonet_Weather.xlsx", sheet_name = "Manhattan")
Manhattan_weather_df["location"] = "Manhattan"
Manhattan_weather_df.rename(columns={'Date / Time': 'datetime'}, inplace=True)
Manhattan_weather_df['datetime'] = pd.to_datetime(Manhattan_weather_df['datetime'], errors='coerce').dt.tz_localize(None)
Manhattan_weather_df.head()

,datetime,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2],location
0,2021-07-24 06:00:00,21.3,66.5,0.9,348,10,Manhattan
1,2021-07-24 06:05:00,21.4,66.1,1.1,345,12,Manhattan
2,2021-07-24 06:10:00,21.4,66.5,1.3,4,14,Manhattan
3,2021-07-24 06:15:00,21.5,65.4,1.3,5,17,Manhattan
4,2021-07-24 06:20:00,21.5,65.0,1.5,346,19,Manhattan


In [5]:
# Display rows that are duplicates
duplicate_rows_manhattan = Manhattan_weather_df[Manhattan_weather_df.duplicated()]
print(duplicate_rows_manhattan)

Empty DataFrame
Columns: [datetime, Air Temp at Surface [degC], Relative Humidity [percent], Avg Wind Speed [m/s], Wind Direction [degrees], Solar Flux [W/m^2], location]
Index: []


## Joining weather variable and uhi_data

In [6]:
# Load the training data from csv file and display the first few rows to inspect the data
ground_df = pd.read_csv("Training_data_uhi_index.csv")
ground_df.head()

,Longitude,Latitude,datetime,UHI Index
0,-73.919037,40.814292,24-07-2021 15:53,1.034616
1,-73.918978,40.814365,24-07-2021 15:53,1.028125
2,-73.918927,40.814433,24-07-2021 15:53,1.028125
3,-73.918875,40.814500,24-07-2021 15:53,1.025961
4,-73.918827,40.814560,24-07-2021 15:53,1.025961


In [7]:
# Define location bounding boxes
manhattan_lower_left = (40.70, -74.02)
manhattan_upper_right = (40.88, -73.92)

bronx_lower_left = (40.79, -73.93)
bronx_upper_right = (40.91, -73.76)

def assign_location(lat, lon):
    if (manhattan_lower_left[0] <= lat <= manhattan_upper_right[0]) and \
       (manhattan_lower_left[1] <= lon <= manhattan_upper_right[1]):
        return "Manhattan"
    elif (bronx_lower_left[0] <= lat <= bronx_upper_right[0]) and \
         (bronx_lower_left[1] <= lon <= bronx_upper_right[1]):
        return "Bronx"
    else:
        return "Outside Range"

# Apply function to assign location labels
ground_df['location'] = ground_df.apply(lambda row: assign_location(row['Latitude'], row['Longitude']), axis=1)

# Convert 'datetime' column to datetime format
ground_df['datetime'] = pd.to_datetime(ground_df['datetime'])

# Check unique locations
print(ground_df['location'].unique())  # ['Manhattan' 'The Bronx' 'Outside Range']

# Check counts of each location
print(ground_df['location'].value_counts())


['Bronx' 'Manhattan']
location
Manhattan    7167
Bronx        4102
Name: count, dtype: int64


In [8]:
ground_df.head()

,Longitude,Latitude,datetime,UHI Index,location,rounded_time
0,-73.919037,40.814292,2021-07-24 15:53:00,1.034616,Bronx,2021-07-24 15:55:00
1,-73.918978,40.814365,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00
2,-73.918927,40.814433,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00
3,-73.918875,40.814500,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00
4,-73.918827,40.814560,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00


In [9]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Combine weather data into a single DataFrame
weather_df = pd.concat([Bronx_weather_df, Manhattan_weather_df], ignore_index=True)

# Convert datetime to Unix timestamps (numerical format)
weather_df['datetime_unix'] = weather_df['datetime'].astype(int) / 10**9
ground_df['datetime_unix'] = ground_df['datetime'].astype(int) / 10**9

# Prepare features for nearest neighbor matching
# Use the Unix timestamp and location columns as keys for matching
weather_features = weather_df[['datetime_unix', 'location']]
train_features = ground_df[['datetime_unix', 'location']]

# Convert location to numerical values (e.g., Bronx = 0, Manhattan = 1)
weather_features['location'] = weather_features['location'].map({'Bronx': 0, 'Manhattan': 1})
train_features['location'] = train_features['location'].map({'Bronx': 0, 'Manhattan': 1})

# Nearest Neighbor Matching
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(weather_features)
distances, indices = nbrs.kneighbors(train_features)

# Merge weather data into train_data_uhi
# Extract the matched weather data
matched_weather = weather_df.iloc[indices.flatten()].reset_index(drop=True)

# Drop unnecessary columns from matched_weather to avoid duplication
matched_weather = matched_weather.drop(columns=['datetime', 'datetime_unix', 'location'])

# Combine the matched weather data with train_data_uhi
training_data_uhi_combined = pd.concat([ground_df.reset_index(drop=True), matched_weather], axis=1)

# Drop the Unix timestamp column (no longer needed)
training_data_uhi_combined = training_data_uhi_combined.drop(columns=['datetime_unix'])

training_data_uhi_combined.head()

,Longitude,Latitude,datetime,UHI Index,location,rounded_time,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,-73.919037,40.814292,2021-07-24 15:53:00,1.034616,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
1,-73.918978,40.814365,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
2,-73.918927,40.814433,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
3,-73.918875,40.814500,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
4,-73.918827,40.814560,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621


In [10]:
training_data_uhi_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11269 entries, 0 to 11268
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Longitude                    11269 non-null  float64       
 1   Latitude                     11269 non-null  float64       
 2   datetime                     11269 non-null  datetime64[ns]
 3   UHI Index                    11269 non-null  float64       
 4   location                     11269 non-null  object        
 5   rounded_time                 11269 non-null  datetime64[ns]
 6   Air Temp at Surface [degC]   11269 non-null  float64       
 7   Relative Humidity [percent]  11269 non-null  float64       
 8   Avg Wind Speed [m/s]         11269 non-null  float64       
 9   Wind Direction [degrees]     11269 non-null  int64         
 10  Solar Flux [W/m^2]           11269 non-null  int64         
dtypes: datetime64[ns](2), float64(6), int64(2

In [11]:
training_data_uhi_combined.to_csv("Training_data_uhi_index_weather.csv", index=False)

In [12]:
training_data_uhi_combined.columns

Index(['Longitude', 'Latitude', 'datetime', 'UHI Index', 'location',
       'rounded_time', 'Air Temp at Surface [degC]',
       'Relative Humidity [percent]', 'Avg Wind Speed [m/s]',
       'Wind Direction [degrees]', 'Solar Flux [W/m^2]'],
      dtype='object')

In [13]:
training_data_uhi_combined.duplicated().sum()

25